# Export VietNet Food Data to Text Format

Xuất toàn bộ dữ liệu VietNet Food thành định dạng văn bản:

```
<number_of_tab> (<level>) {<lemma1>, <lemma2>, ...} [<synset_id>]: <definition>
```


In [ ]:
# Import và xử lí dữ liệu
import sys
import os

# Add components to path
sys.path.append(os.path.join(os.getcwd(), 'components'))

from components.utils_wn import *
from components.class_Node import *
from components.class_NodeFamily import *
import wn

# Initialize lexicon
wn.remove('vietnet-food:1.0')
wn.add('../vietnet_food (thủ công).xml')
lexicon = wn.Wordnet('vietnet-food:1.0')

# Get all root synsets (synsets with no hypernyms)
all_synsets = list(lexicon.synsets())
root_synsets = [syn for syn in all_synsets if not syn.hypernyms()]

print(f"Tổng số synsets: {len(all_synsets)}")
print(f"Số lượng root synsets: {len(root_synsets)}")

# Build tree with hyponym relationship and maximum depth
families = NodeFamily(root_synsets, 'hyponym', max_recursive=100)

# Function to export nodes to text with proper formatting
def export_to_text(nodes, output_file='vietnet_food_export.txt'):
    """
    Export node tree to text file with format:
    <tabs> (<level>) {lemmas} [synset_id]: definition
    """
    lines = []
    
    def traverse_node(node, depth=0):
        # Calculate level from root (node._level is absolute level from root)
        level = node._level - 1  # Convert to 0-based level
        
        # Create tabs for indentation
        tabs = '\t' * level
        
        # Get lemmas
        lemmas = node._lemmas
        
        # Get synset ID
        synset_id = node._synset.id
        
        # Get definition
        definition = node._definition
        
        # Format: <tabs> (<level>) {lemmas} [synset_id]: definition
        line = f"{tabs}({level}) {{{lemmas}}} [{synset_id}]: {definition}"
        lines.append(line)
        
        # Recursively traverse children
        if node.children:
            for child in node.children:
                traverse_node(child, depth + 1)
    
    # Traverse all root nodes
    for node in nodes:
        traverse_node(node)
    
    # Write to file
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write('\n'.join(lines))
    
    print(f"\n✅ Đã xuất {len(lines)} dòng dữ liệu vào file: {output_file}")
    print(f"\n📄 Preview (20 dòng đầu):")
    print("=" * 80)
    for i, line in enumerate(lines[:20]):
        print(line)
    print("=" * 80)
    
    return lines

# Export data
exported_lines = export_to_text(families.nodes, 'vietnet_food_export.txt')

print(f"\n📊 Thống kê:")
print(f"   - Tổng số dòng đã xuất: {len(exported_lines)}")
print(f"   - File được lưu tại: vietnet_food_export.txt")
print(f"\n💡 Bạn có thể mở file này bằng Word hoặc Google Docs để xem toàn bộ dữ liệu.")


Read [##############################] (4671/4671) 
Added vietnet-food:1.0 (VietNet Vietnamese Food Lexicon (All))ionssours



Tổng số synsets: 590
Số lượng root synsets: 2

✅ Đã xuất 590 dòng dữ liệu vào file: vietnet_food_export.txt

📄 Preview (20 dòng đầu):
(0) {thức ăn, chất} [vietnet-food-00045020-n]: thứ có chứa chất dinh dưỡng và có thể tiêu hóa được, dùng để nuôi sống con người và động vật, nói chung
	(1) {thực phẩm} [vietnet-food-00045060-n]: các thứ dùng để chế biến thành món ăn, như thịt, cá, trứng, v.v. [nói khái quát]; phân biệt với lương thực
		(2) {nông sản, nông phẩm} [vietnet-food-00034540-n]: sản phẩm nông nghiệp, như gạo, thịt, trứng, rau, hoa quả, v.v. [nói khái quát]
			(3) {súc sản} [vietnet-food-00040488-n]: sản phẩm thịt gia súc
				(4) {cầy} [vietnet-food-00006363-n]: chó, về mặt để ăn thịt
				(4) {giò} [vietnet-food-00018716-n]: cẳng chân gia súc hoặc gia cầm
				(4) {nạc} [vietnet-food-00030509-n]: thịt không có mỡ
				(4) {thăn} [vietnet-food-00042705-n]: phần thịt toàn nạc được lọc ra ở phần lưng của súc vật khi mổ thịt
				(4) {nọng} [vietnet-food-00034330-n]: khoanh thịt cắt ra 

In [ ]:
# THỐNG KÊ CHÍNH XÁC - ĐẾM THEO SỐ LẦN XUẤT HIỆN (KHÔNG LOẠI BỎ TRÙNG LẶP)
import pandas as pd
from collections import defaultdict

def analyze_tree_structure_exact(nodes, branch_name=""):
    """
    Phân tích cấu trúc phân cấp của cây - ĐẾM CHÍNH XÁC theo số lần xuất hiện
    KHÔNG loại bỏ mục từ hoặc định nghĩa trùng nhau
    """
    level_stats = defaultdict(lambda: {'lemmas_count': 0, 'definitions_count': 0, 'synsets_count': 0})
    
    def traverse_and_count(node):
        level = node._level - 1  # Convert to 0-based
        
        # Đếm số synsets (mỗi node = 1 synset)
        level_stats[level]['synsets_count'] += 1
        
        # Đếm số định nghĩa (mỗi synset có 1 định nghĩa, đếm tất cả kể cả trùng)
        if node._definition:
            level_stats[level]['definitions_count'] += 1
        
        # Đếm số mục từ (lemmas) - đếm tất cả các lemma kể cả trùng
        lemmas_list = node._lemmas.split(', ')
        level_stats[level]['lemmas_count'] += len(lemmas_list)
        
        # Đệ quy với các con
        if node.children:
            for child in node.children:
                traverse_and_count(child)
    
    # Duyệt tất cả các node gốc
    for node in nodes:
        traverse_and_count(node)
    
    # Tạo bảng thống kê
    max_level = max(level_stats.keys()) if level_stats else 0
    
    data = []
    total_lemmas = 0
    total_definitions = 0
    total_synsets = 0
    
    for level in range(max_level + 1):
        if level in level_stats:
            num_lemmas = level_stats[level]['lemmas_count']
            num_definitions = level_stats[level]['definitions_count']
            num_synsets = level_stats[level]['synsets_count']
        else:
            num_lemmas = 0
            num_definitions = 0
            num_synsets = 0
        
        data.append({
            'Tầng': level,
            'Số mục từ danh từ': num_lemmas,
            'Số định nghĩa': num_definitions
        })
        
        total_lemmas += num_lemmas
        total_definitions += num_definitions
        total_synsets += num_synsets
    
    # Thêm dòng tổng
    data.append({
        'Tầng': 'Tổng',
        'Số mục từ danh từ': total_lemmas,
        'Số định nghĩa': total_definitions
    })
    
    df = pd.DataFrame(data)
    
    return df, total_lemmas, total_definitions, total_synsets

# Phân tích toàn bộ cây
print("="*80)
print("THỐNG KÊ CẤU TRÚC PHÂN CẤP HỆ THỐNG NHÃN NGỮ NGHĨA")
print("(ĐẾM CHÍNH XÁC - KHÔNG LOẠI BỎ TRÙNG LẶP)")
print("="*80)

# Phân tích theo từng nhánh gốc
for idx, root_node in enumerate(families.nodes):
    root_lemmas = root_node._lemmas
    root_definition = root_node._definition
    
    print(f"\n📊 Bảng {idx + 10}. Cấu trúc phân cấp hệ thống nhãn ngữ nghĩa cho nhánh \"{root_lemmas}\"")
    print("-" * 80)
    
    # Phân tích nhánh này
    df, total_lemmas, total_defs, total_synsets = analyze_tree_structure_exact([root_node], root_lemmas)
    
    # Hiển thị bảng
    print(df.to_string(index=False))
    print("-" * 80)
    
    # Tìm max level
    max_level = df[df['Tầng'] != 'Tổng']['Tầng'].max()
    
    # Tìm các tầng có số mục từ nhiều nhất (không tính tầng 0 và Tổng)
    df_levels = df[df['Tầng'] != 'Tổng'].copy()
    df_levels = df_levels[df_levels['Tầng'] != 0]
    
    if len(df_levels) > 0:
        # Sắp xếp theo số mục từ giảm dần
        df_sorted = df_levels.sort_values('Số mục từ danh từ', ascending=False)
        top_levels = df_sorted.head(2)
        
        summary_parts = []
        for _, row in top_levels.iterrows():
            level = int(row['Tầng'])
            num_lemmas = int(row['Số mục từ danh từ'])
            num_defs = int(row['Số định nghĩa'])
            summary_parts.append(f"tầng {level} ({num_lemmas} mục từ, {num_defs} định nghĩa)")
        
        summary = f"Nhánh \"{root_lemmas}\" bao gồm {total_lemmas} mục từ với {total_defs} định nghĩa, "
        summary += f"tập trung ở {' và '.join(summary_parts)}"
        summary += f" và kết thúc ở tầng {max_level}."
        
        print(f"\n💡 {summary}")
    
    print("\n")

# Thống kê tổng thể
print("="*80)
print("📈 THỐNG KÊ TỔNG THỂ")
print("="*80)

df_total, total_lemmas_all, total_defs_all, total_synsets_all = analyze_tree_structure_exact(families.nodes, "Toàn bộ")
print(df_total.to_string(index=False))
print("="*80)
print(f"\n✅ Tổng cộng: {total_lemmas_all} mục từ danh từ | {total_defs_all} định nghĩa")
print(f"📚 Tổng số synsets: {len(all_synsets)}")
print(f"🌳 Số nhánh gốc: {len(root_synsets)}")
print("\n")


THỐNG KÊ CẤU TRÚC PHÂN CẤP HỆ THỐNG NHÃN NGỮ NGHĨA

📊 Bảng 10. Cấu trúc phân cấp hệ thống nhận ngữ nghĩa cho nhánh "thức ăn, chất"
--------------------------------------------------------------------------------
Tầng  Số mục từ danh từ  Số định nghĩa
   0                  2              1
   1                  3              3
   2                 43             37
   3                154            121
   4                124            103
   5                  5              5
Tổng                331            270
--------------------------------------------------------------------------------

💡 Nhánh "thức ăn, chất" bao gồm 331 mục từ với 270 định nghĩa, tập trung ở tầng 3 (154 mục từ, 121 định nghĩa) và tầng 4 (124 mục từ, 103 định nghĩa) và kết thúc ở tầng 5.



📊 Bảng 11. Cấu trúc phân cấp hệ thống nhận ngữ nghĩa cho nhánh "thức ăn, thức"
--------------------------------------------------------------------------------
Tầng  Số mục từ danh từ  Số định nghĩa
   0                